Define Variables and imports

In [152]:
import yfinance as yf
import datetime as dt
import pandas as pd
import numpy as np
import time

# Set up the start and end dates for the data
start_date = '2020-01-02'
end_date = '2023-04-04'

# Set up the ticker symbols for TQQQ and SQQQ
tqqq_symbol = 'TQQQ'
sqqq_symbol = 'SQQQ'

portfolio_df = pd.DataFrame(columns=['Week', 'TQQQ Call Buy Price', 'TQQQ Call Sell Price', 'SQQQ Put Buy Price', 'SQQQ Put Sell Price', 'Total Cost', 'Total P/L'])
# Set up the options expiration date as the third Friday of each month
def get_third_friday(year, month):
    d = dt.date(year, month, 15)
    while d.weekday() != 4:
        d += dt.timedelta(1)
    return d

# Set up the strike prices and width of the spreads
spread_width = 0.05
tqqq_strike_price = 0
sqqq_strike_price = 0

# Set up the initial portfolio value
initial_portfolio_value = 10000


Define a function to calculate the profit/loss of a trade

In [153]:
def calculate_profit_loss(quantity, buy_price, sell_price):
    return quantity * (sell_price - buy_price)

# Set up an empty list to store the portfolio values over time
portfolio_values = []

Download the historical data for TQQQ and SQQQ

In [154]:
tqqq_data = yf.download(tqqq_symbol, start=start_date, end=end_date)
sqqq_data = yf.download(sqqq_symbol, start=start_date, end=end_date)

spread_width = 0.05  # Set the spread width for the call and put spreads
tqqq_data['Strike'] = round(tqqq_data['Close'] / spread_width) * spread_width
sqqq_data['Strike'] = round(sqqq_data['Close'] / spread_width) * spread_width



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [155]:
#display Data
tqqq_data

,Open,High,Low,Close,Adj Close,Volume,Strike
Date,,,,,,,
2020-01-02,22.177500,22.709999,22.055000,22.709999,22.461794,65536000,22.70
2020-01-03,21.822500,22.500000,21.809999,22.092501,21.851044,72590000,22.10
2020-01-06,21.567499,22.520000,21.487499,22.512501,22.266453,64047600,22.50
2020-01-07,22.535000,22.695000,22.290001,22.485001,22.239254,53849600,22.50
2020-01-08,22.490000,23.320000,22.382500,23.002501,22.751099,79582400,23.00
...,...,...,...,...,...,...,...
2023-03-28,25.139999,25.139999,24.290001,24.870001,24.870001,118020700,24.85
2023-03-29,25.760000,26.370001,25.570000,26.230000,26.230000,119365300,26.25
2023-03-30,26.820000,27.090000,26.490000,26.940001,26.940001,130150000,26.95


In [156]:
sqqq_data

,Open,High,Low,Close,Adj Close,Volume,Strike
Date,,,,,,,
2020-01-02,546.000000,549.250000,532.500000,532.500000,526.091980,644972,532.50
2020-01-03,553.250000,553.500000,537.250000,546.750000,540.170532,728464,546.75
2020-01-06,559.750000,561.750000,536.250000,536.250000,529.796875,627520,536.25
2020-01-07,535.750000,541.750000,532.000000,536.500000,530.043823,442268,536.50
2020-01-08,537.000000,539.500000,517.250000,524.500000,518.188232,730600,524.50
...,...,...,...,...,...,...,...
2023-03-28,33.220001,34.320000,33.220001,33.549999,33.549999,110031400,33.55
2023-03-29,32.340000,32.619999,31.540001,31.760000,31.760000,120130400,31.75
2023-03-30,31.020000,31.410000,30.680000,30.879999,30.879999,115341700,30.90


Loop over each month in the data and buy the call and put spreads on the third Friday


In [160]:
# Loop over each month in the data and buy the call and put spreads on the third Friday
for year in range(int(start_date[:4]), int(end_date[:4])+1):
    for month in range(1, 13):
        if year == int(start_date[:4]) and month < int(start_date[5:7]):
            continue
        if year == int(end_date[:4]) and month > int(end_date[5:7]):
            continue

        # Get the expiration date for this month
        expiration_date = get_third_friday(year, month)

        # Only execute the trade on Fridays
        if expiration_date.weekday() != 4:
            continue

        # Get the data for this month
        # tqqq_month_data = tqqq_data.loc[start_date:end_date][str(year)+'-'+str(month)]
        # sqqq_month_data = sqqq_data.loc[start_date:end_date][str(year)+'-'+str(month)]
       # tqqq_month_data = tqqq_data.loc[start_date:end_date][pd.to_datetime(str(year)+'-'+str(month))]
        #sqqq_month_data = sqqq_data.loc[start_date:end_date][pd.to_datetime(str(year)+'-'+str(month))]
        # Get the data for this month
        month_start = pd.to_datetime(str(year)+'-'+str(month)+'-01')
        month_end = month_start + pd.DateOffset(months=1) - pd.Timedelta(days=1)
        tqqq_month_data = tqqq_data.loc[month_start:month_end]
        sqqq_month_data = sqqq_data.loc[month_start:month_end]
        print(tqqq_month_data)
        print(sqqq_month_data)

        #Find the ATM strike price for TQQQ and SQQQ
        tqqq_close = tqqq_month_data['Close']
        tqqq_strike_price = round(tqqq_close / spread_width) * spread_width
        sqqq_close = sqqq_month_data['Close']
        sqqq_strike_price = round(sqqq_close / spread_width) * spread_width
         # Buy the call and put spreads at the mid-market rate
        tqqq_ask = tqqq_month_data['High'][tqqq_month_data['Strike'] == tqqq_strike_price].values[0]
        tqqq_bid = tqqq_month_data['Low'][tqqq_month_data['Strike'] == tqqq_strike_price].values[0]
        tqqq_mid = (tqqq_ask + tqqq_bid) / 2
        sqqq_ask = sqqq_month_data['High'][sqqq_month_data['Strike'] == sqqq_strike_price].values[0]
        sqqq_bid = sqqq_month_data['Low'][sqqq_month_data['Strike'] == sqqq_strike_price].values[0]
        sqqq_mid = (sqqq_ask + sqqq_bid) / 2
        tqqq_call_buy_price = tqqq_mid
        tqqq_call_sell_price = tqqq_month_data['Low'][tqqq_month_data['Strike'] == tqqq_strike_price + spread_width].values
        if len(tqqq_call_sell_price) > 0:
            tqqq_call_sell_price = tqqq_call_sell_price[0]
        else:
            # handle the case where the filter did not match any rows
            tqqq_call_sell_price = 0.0  # set some default value        
        tqqq_put_sell_price = sqqq_month_data['Low'][sqqq_month_data['Strike'] == sqqq_strike_price - spread_width].values
        if len(tqqq_put_sell_price) > 0:
            tqqq_put_sell_price = tqqq_put_sell_price[0]
        else:
            # handle the case where the filter did not match any rows
            tqqq_put_sell_price = 0.0  # set some default value 
           
        sqqq_call_sell_price = sqqq_month_data['Low'][sqqq_month_data['Strike'] == sqqq_strike_price + spread_width].values
        if len(sqqq_call_sell_price) > 0:
            sqqq_call_sell_price = sqqq_call_sell_price[0]
        else:
            # handle the case where the filter did not match any rows
            sqqq_call_sell_price = 0.0  # set some default value       
        
        tqqq_put_buy_price = tqqq_mid

        sqqq_put_sell_price = sqqq_month_data['Low'][sqqq_month_data['Strike'] == sqqq_strike_price - spread_width].values
        if len(sqqq_put_sell_price) > 0:
            sqqq_put_sell_price = sqqq_put_sell_price[0]
        else:
            # handle the case where the filter did not match any rows
            sqqq_put_sell_price = 0.0  # set some default value 
        #tqqq_put_sell_price = sqqq_month_data['Low'][sqqq_month_data['Strike'] == sqqq_strike_price - spread_width].values[0] 
        sqqq_put_buy_price = sqqq_mid      


        # Record the trade in the portfolio
        tqqq_call_quantity = int(initial_portfolio_value / tqqq_call_buy_price)
        tqqq_call_profit_loss = calculate_profit_loss(tqqq_call_quantity, tqqq_call_buy_price, tqqq_call_sell_price)
        tqqq_put_quantity = int(initial_portfolio_value / tqqq_put_buy_price)
        tqqq_put_profit_loss = calculate_profit_loss(tqqq_put_quantity, tqqq_put_buy_price, tqqq_put_sell_price)
        total_cost = (tqqq_call_buy_price + sqqq_put_buy_price)
        # Calculate the total profit/loss for this trade
        total_profit_loss = tqqq_call_profit_loss + tqqq_put_profit_loss

        # Update the portfolio value
        initial_portfolio_value += total_profit_loss
        portfolio_values.append(initial_portfolio_value)    
        # Iterate over weeks
        # # Append row to portfolio dataframe
        i= 0
        portfolio_df = pd.concat([portfolio_df, pd.DataFrame({'Week': expiration_date, 
                                        'TQQQ Call Buy Price': tqqq_call_buy_price, 
                                        'TQQQ Call Sell Price': tqqq_call_sell_price, 
                                        'SQQQ Put Buy Price': sqqq_bid, 
                                        'SQQQ Put Sell Price': sqqq_put_sell_price, 
                                        'Total Cost': total_cost, 
                                        'Total P/L': total_profit_loss}, index=[i+1])])

        # Print the trade details
        print(f"Trade Date: {expiration_date}\n"
              f"TQQQ Call Buy Price: {tqqq_call_buy_price:.2f}\n"
              f"TQQQ Call Sell Price: {tqqq_call_sell_price:.2f}\n"
              f"TQQQ Call Quantity: {tqqq_call_quantity}\n"
              f"TQQQ Call P/L: {tqqq_call_profit_loss:.2f}\n"
              f"SQQQ Put Buy Price: {tqqq_put_buy_price:.2f}\n"
              f"SQQQ Put Sell Price: {tqqq_put_sell_price:.2f}\n"
              f"SQQQ Put Quantity: {tqqq_put_quantity}\n"
              f"SQQQ Put P/L: {tqqq_put_profit_loss:.2f}\n"
              f"Total P/L: {total_profit_loss:.2f}\n"
              f"Portfolio Value: {initial_portfolio_value:.2f}\n"
              f"-----------------------------------------")

        # Wait for a few seconds before executing the next trade
        time.sleep(5)
# Print the final portfolio value
print(f"Final Portfolio Value: {initial_portfolio_value:.2f}")

                 Open       High        Low      Close  Adj Close     Volume  \
Date                                                                           
2020-01-02  22.177500  22.709999  22.055000  22.709999  22.461794   65536000   
2020-01-03  21.822500  22.500000  21.809999  22.092501  21.851044   72590000   
2020-01-06  21.567499  22.520000  21.487499  22.512501  22.266453   64047600   
2020-01-07  22.535000  22.695000  22.290001  22.485001  22.239254   53849600   
2020-01-08  22.490000  23.320000  22.382500  23.002501  22.751099   79582400   
2020-01-09  23.557501  23.715000  23.180000  23.582500  23.324760   72711600   
2020-01-10  23.847500  23.867500  23.282499  23.389999  23.134363   68802000   
2020-01-13  23.682501  24.219999  23.577499  24.207500  23.942928   58279200   
2020-01-14  24.145000  24.287500  23.805000  23.924999  23.663515   72868800   
2020-01-15  23.945000  24.282499  23.705000  23.940001  23.678352   66501600   
2020-01-16  24.277500  24.622499  24.132

Plot the portfolio values over time

In [ ]:
# Plot the portfolio values over time
portfolio_df = pd.DataFrame(portfolio_values, columns=['Value'])
portfolio_df.plot(figsize=(10,5), title='Portfolio Value Over Time')
# print portfolio
portfolio 